In [ ]:
import os
import re
import pandas as pd
from gimmemotifs.preprocessing import combine_peaks
import subprocess as sp

Matplotlib is building the font cache; this may take a moment.
2021-05-03 15:27:16,139 - INFO - No config found.
2021-05-03 15:27:16,141 - INFO - Creating new config.
2021-05-03 15:27:16,172 - INFO - Using included version of MDmodule.
2021-05-03 15:27:45,184 - INFO - Using system version of MEME.
2021-05-03 15:28:03,943 - INFO - Using system version of MEMEW.
2021-05-03 15:28:37,520 - INFO - Using system version of DREME.
2021-05-03 15:28:56,648 - INFO - Using system version of Weeder.
2021-05-03 15:29:14,700 - INFO - Using system version of GADEM.
2021-05-03 15:29:14,702 - INFO - Using included version of MotifSampler.
2021-05-03 15:29:32,678 - INFO - Using system version of Trawler.
2021-05-03 15:29:32,680 - INFO - Using included version of Improbizer.
2021-05-03 15:29:32,681 - INFO - Using included version of BioProspector.
2021-05-03 15:29:32,683 - INFO - Using included version of Posmo.
2021-05-03 15:29:32,684 - INFO - Using included version of ChIPMunk.
2021-05-03 15:29:32,685 -

In [ ]:
#load the table to convert the chromosome names to the USCS format
CRCH38_ensembl2UCSC = pd.read_csv('/scratch/bacint/jdeleuw/KC_ATAC/analysis/bigBed/CRCH38_ensembl2UCSC.txt', sep ='\t',header = 0, index_col=False)

#make dictionary which can be used to convert ensembl to UCSC format 
ensembl_to_UCSC = {}
UCSC_to_ensembl = {}
for row in CRCH38_ensembl2UCSC.itertuples():
    ensembl_to_UCSC[row[1]] = row[2]
    UCSC_to_ensembl[row[2]] = row[1]

In [39]:
paths = pd.read_csv('/scratch/bacint/jdeleuw/ananse/ananse_paths.csv', sep = ',', header = 0, index_col = 0)

In [ ]:
paths

# Ananse enhancer

In [5]:
#Copy narrowPeak files to the ananse map and change the name to the day of differentiation
for narrowPeak, output_dir in zip(paths['narrowPeak'], paths['narrowPeak_descriptive_name']):
    sp.check_call(f'cp {narrowPeak} '
                  f'{output_dir}',
                  shell = True)

In [6]:
#Copy bigWig files to the ananse map and change the name to the day of differentiation
for bigWig, output_dir in zip(paths['bigWig'], paths['bigWig_descriptive_name']):
    sp.check_call(f'cp {bigWig} '
                  f'{output_dir}',
                  shell = True)

In [7]:
#select the overlapping 200 bp peaks
genome_path_size = '/scratch/bacint/jdeleuw/genomes/hg38/hg38.fa.sizes'

peak_files = []

for index, cell_type in paths.iterrows():
    peak_files.extend(paths.narrowPeak_descriptive_name[index].split(' '))

all_peaks_merged = combine_peaks(peak_files,
                                 genome_path_size,
                                 int(200),
                                 True)
#extend each summit to 200bp
all_peaks_merged.start = all_peaks_merged.start.astype(int) - 100
all_peaks_merged.end = all_peaks_merged.end.astype(int) + 100

all_peaks_merged = all_peaks_merged[~all_peaks_merged.chrom.str.contains("GL|Un|KI|MT|X|Y")]

all_peaks_merged.to_csv(
    '/scratch/bacint/jdeleuw/ananse/enhancer_data/peaks_merged.tsv',
    sep="\t",
    header=False,
    index=False,
)

all_peaks_merged['peak'] = all_peaks_merged.apply(lambda row: ' '.join(row.values.astype(str)), axis=1)

In [ ]:
#specify bigWig paths
bigWig_paths = []
bigWig_paths.extend(paths['bigWig_descriptive_name'])

#specify output directory
output_enhancer_activity = []
output_enhancer_activity.extend(paths['bigWigSummary_output'].dropna())
output_dir_slice = 0

#Calculate the max score for every bw file
for index, bigWig in zip(paths.index, bigWig_paths):  
    #for duplicate A, which should not be written to a file yet
    if index.endswith('A') or index.endswith('2'):
        output = []
        for line in all_peaks_merged['peak']:
            output_bigWigSummary = sp.run(f'bigWigSummary '
                                          f'{bigWig} '
                                          f'-type=max '
                                          f'{line} 1',
                                          stdout = sp.PIPE,
                                          shell = True) 
            max_score = str(output_bigWigSummary.stdout).strip("\n'")[2:-2].strip()
            row = line.replace(' ','\t') + '\t' + max_score
            output.append(row)    
        peak_intensity_temp = pd.DataFrame(output)
        #split the row in the rows chr, start, end and intensity
        peak_intensity = peak_intensity_temp.loc[:,0].str.split('\t', expand = True)
        peak_intensity = peak_intensity.rename(columns = {0:'chrom', 1:'start', 2:'end', 3:'intensity_A'})

        #convert the data type of start and end to float, so it can be properly sorted
        cols =  ['start', 'end', 'intensity_A']
        peak_intensity[cols] = peak_intensity[cols].apply(pd.to_numeric, errors='coerce')
        sorted_peak_intensity_A = peak_intensity.sort_values(by = ['chrom', 'start', 'end'])

    #for duplicate B, for which the average will be calculated with duplicate A and written to a file
    else:
        output = []
        for line in all_peaks_merged['peak']:
            output_bigWigSummary = sp.run(f'bigWigSummary '
                                          f'{bigWig} '
                                          f'-type=max '
                                          f'{line} 1',
                                          stdout = sp.PIPE,
                                          shell = True) 
            max_score = str(output_bigWigSummary.stdout).strip("\n'")[2:-2].strip()
            row = line.replace(' ','\t') + '\t' + max_score
            output.append(row)    
        peak_intensity_temp = pd.DataFrame(output)
        #split the row in the rows chr, start, end and intensity
        peak_intensity = peak_intensity_temp.loc[:,0].str.split('\t', expand = True)
        peak_intensity = peak_intensity.rename(columns = {0:'chrom', 1:'start', 2:'end', 3:'intensity_B'})

        #convert the data type of start and end to float, so it can be properly sorted
        cols =  ['start', 'end', 'intensity_B']
        peak_intensity[cols] = peak_intensity[cols].apply(pd.to_numeric, errors='coerce')
        sorted_peak_intensity_B = peak_intensity.sort_values(by = ['chrom', 'start', 'end'])

        #calculate average of max score between two duplicates
        sorted_peak_intensity_A['intensity_B'] = sorted_peak_intensity_B['intensity_B']
        sorted_peak_intensity_A['intensity_avg'] = sorted_peak_intensity_A[['intensity_A', 'intensity_B']].mean(axis = 1).round()
        avg_enhancer_intensity = sorted_peak_intensity_A.drop(sorted_peak_intensity_A[['intensity_A', 'intensity_B']], axis=1)

        avg_enhancer_intensity.to_csv(output_enhancer_activity[output_dir_slice], sep = '\t', header = False, index = False)
        output_dir_slice += 1

In [11]:
#sort the enhancer intensity files
#specify input enhancer activity paths
enhancer_activity = []
enhancer_activity.extend(paths['bigWigSummary_output'].dropna())

#specify output directory
output_paths = []
output_paths.extend(paths['bed_sorted'].dropna())

for bed_file, output_dir in zip(enhancer_activity, output_paths):
    sp.check_call(f'LC_COLLATE=C sort -k1,1 -k2,2n '
                  f'{bed_file} '
                  f'> {output_dir}',
                  shell=True)

In [ ]:
#delete the config file of gimmemotifs
sp.check_call(f'rm -r '
              f'/mbshome/jdeleuw/.config/gimmemotifs/',
              shell = True)

# Ananse binding

In [ ]:
#run ananse binding
#specify the enhancer intensity paths constructed in the last step
enhancer_intensity = []
enhancer_intensity.extend(paths['bed_sorted'].dropna())

#specify the output directory paths
output_paths = []
output_paths.extend(paths['binding_output'].dropna())

genome = '/scratch/bacint/jdeleuw/genomes/hg38/hg38.fa'

#specify the log output dir
binding_log = []
binding_log.extend(paths['binding_log'].dropna())

for bed, output_dir, log in zip(enhancer_intensity, output_paths, binding_log):
    sp.check_call(f'nice -15 '
                  f'ananse binding '
                  f'-r {bed} '
                  f'-o {output_dir} '
                  f'-g {genome} '
                  f'-t ATAC '
                  f'2> {log}',
                  shell = True)

# Ananse network

In [9]:
#run ananse network
#specify TPM_paths
TPM_duplicate_A = []
TPM_duplicate_B = []

for index, values in paths.iterrows():
    if index.endswith('A') or index.endswith('KC2'):
        TPM_duplicate_A.append(values.TPM)
    else:
        TPM_duplicate_B.append(values.TPM)
#specify binding paths
binding_paths = []
binding_paths.extend(paths['binding_output'].dropna())
#specify network output
network_paths = []
network_paths.extend(paths.network_output.dropna())
#specify log output
network_log = []
network_log.extend(paths.network_log.dropna())     

bed_file_network = '/scratch/bacint/jdeleuw/ananse/genome/hg38_genes.bed'
genome = '/scratch/bacint/jdeleuw/genomes/hg38/hg38.fa'

for TPM_A, TPM_B, binding, output_dir, log in zip(TPM_duplicate_A, TPM_duplicate_B,
                                                  binding_paths,
                                                  network_paths,
                                                  network_log):
    sp.check_call(f'nice -15 ananse network '
                  f'-e {TPM_A} {TPM_B} '
                  f'-b {binding} '
                  f'-o {output_dir} '
                  f'-a {bed_file_network} '
                  f'-g {genome} '
                  f'2> {log}',
                  shell = True)

In [10]:
paths = pd.read_csv('/scratch/bacint/jdeleuw/ananse/ananse_paths.csv', sep = ',', header = 0, index_col = 0)

# Ananse influence

In [ ]:
#run ananse influence
#Specify DEgenes paths
DEgenes_paths = []
DEgenes_paths.extend(paths.Deseq.dropna())
#Specify the network input paths
network_paths = []
network_paths.extend(paths.network_output.dropna())
#Specify influence output
influence_paths = []
influence_paths.extend(paths.influence_output.dropna())
#Specify log output
influence_log = []
influence_log.extend(paths.influence_log.dropna())

for network, DEgenes, output_dir, log in zip(network_paths[1:], 
                                             DEgenes_paths, 
                                             influence_paths, 
                                             influence_log):
    sp.check_call(f'nice -15 ananse influence '
                  f'-s /scratch/bacint/jdeleuw/ananse/network_data/Day0_network.txt '
                  f'-t {network} '
                  f'-d {DEgenes} '
                  f'-p '
                  f'-o {output_dir} ' 
                  f'2> {log}',
                  shell = True) 

In [ ]:
#run ananse influence with day 0 as the target
#Specify DEgenes paths
rev_DEgenes_paths = []
rev_DEgenes_paths.extend(paths.reverse_Deseq.dropna())
#Specify the network input paths
network_paths = []
network_paths.extend(paths.network_output.dropna())
#Specify influence output
rev_influence_paths = []
rev_influence_paths.extend(paths.reverse_influence_output.dropna())
#Specify log output
rev_influence_log = []
rev_influence_log.extend(paths.reverse_influence_log.dropna())

for network, DEgenes, output_dir, log in zip(network_paths[1:], 
                                             rev_DEgenes_paths, 
                                             rev_influence_paths, 
                                             rev_influence_log):
    sp.check_call(f'nice -15 ananse influence '
                  f'-s {network} '
                  f'-t /scratch/bacint/jdeleuw/ananse/network_data/Day0_network.txt '
                  f'-d {DEgenes} '
                  f'-p '
                  f'-o {output_dir} ' 
                  f'2> {log}',
                  shell = True)

In [23]:
#run ananse influence in chronological order
#Specify DEgenes paths
chron_DEgenes_paths = []
chron_DEgenes_paths.extend(paths.chron_Deseq.dropna())
#Specify the network input paths
network_paths = []
network_paths.extend(paths.network_output.dropna())
#Specify influence output
chron_influence_paths = []
chron_influence_paths.extend(paths.chron_influence_output.dropna())
#Specify log output
chron_influence_log = []
chron_influence_log.extend(paths.chron_influence_log.dropna())

for source_network, target_network, DEgenes, output_dir, log in zip(network_paths,
                                                                    network_paths[1:], 
                                                                    chron_DEgenes_paths, 
                                                                    chron_influence_paths, 
                                                                    chron_influence_log):
    sp.check_call(f'nice -15 ananse influence '
                  f'-s {source_network} '
                  f'-t {target_network} '
                  f'-d {DEgenes} '
                  f'-p '
                  f'-o {output_dir} ' 
                  f'2> {log}',
                  shell = True)

# Average comparisons

In [34]:
#Calculate the enhancer activity
#specify paths to enhancer activity files
enhancer_activity_paths = []
enhancer_activity_paths.extend(paths['bigWigSummary_output'].dropna())
#load enhancer activity files
variable_names = [day + '_enhancer' for day in paths.index]
for path, name in zip(enhancer_activity_paths, variable_names):
    globals()[name] = pd.read_csv(path, sep ='\t', header = None, index_col=False)

#calculate avg enhancer intensity
avg_intensity = (Day0_enhancer.loc[:,3] + Day07_enhancer.loc[:,3] + 
                 Day14_enhancer.loc[:,3] + Day21_enhancer.loc[:,3] + 
                 Day43_enhancer.loc[:,3] + Day60_enhancer.loc[:,3] + 
                 KC_enhancer.loc[:,3]) // 7

#merge the enhancer activities with the peak location
cols = [0,1,2]
average_bed = Day0_enhancer[cols].join(avg_intensity)
average_bed.to_csv('/scratch/bacint/jdeleuw/ananse/enhancer_data/enhancer_intensity/average_enhancer_intensity.tsv', sep = '\t', header = False, index = False)

In [35]:
# run ananse binding
sp.check_call(f'nice -15 '
              f'ananse binding '
              f'-r /scratch/bacint/jdeleuw/ananse/enhancer_data/enhancer_intensity_sorted/average_enhancer_activity.tsv '
              f'-o /scratch/bacint/jdeleuw/ananse/binding_data/binding_avg.txt '
              f'-g {genome} '
              f'-t ATAC '
              f'2> /scratch/bacint/jdeleuw/ananse/logs/binding_avg.log',
              shell = True)

0

In [38]:
# calculate average TPM
variable_names = [day + '_TPM' for day in paths.index]
for path, name in zip(paths.TPM, variable_names):
    globals()[name] = pd.read_csv(path, sep ='\t', header = 0, index_col=False)
    
avg_tpm = (Day0A_TPM.iloc[:,1] + Day0B_TPM.iloc[:,1] + 
           Day07A_TPM.iloc[:,1] + Day07B_TPM.iloc[:,1] +
           Day14A_TPM.iloc[:,1] + Day14B_TPM.iloc[:,1] +
           Day21A_TPM.iloc[:,1] + Day21B_TPM.iloc[:,1] +
           Day43A_TPM.iloc[:,1] + Day43B_TPM.iloc[:,1] +
           Day60A_TPM.iloc[:,1] + Day60B_TPM.iloc[:,1] +
           KC2_TPM.iloc[:,1] +  KC3_TPM.iloc[:,1]) // 14
average_TPM = Day0A_TPM.drop(['tpm'], axis=1).join(avg_tpm)
average_TPM.to_csv('/scratch/bacint/jdeleuw/ananse/expression_data/TPM/avg_TPM.tsv', sep = '\t', header = True, index = False)

In [ ]:
#run ananse network
bed_file_network = '/scratch/bacint/jdeleuw/ananse/genome/hg38_genes.bed'
genome = '/scratch/bacint/jdeleuw/genomes/hg38/hg38.fa'

sp.check_call(f'nice -15 ananse network '
              f'-e /scratch/bacint/jdeleuw/ananse/expression_data/TPM/avg_TPM.tsv '
              f'-b /scratch/bacint/jdeleuw/ananse/binding_data/binding_avg.txt '
              f'-o /scratch/bacint/jdeleuw/ananse/network_data/avg_network.txt '
              f'-a {bed_file_network} '
              f'-g {genome} '
              f'2> /scratch/bacint/jdeleuw/ananse/logs/avg_network.log ',
              shell = True
              )

In [ ]:
#run ananse influence by comparing everything to the average
#Specify the network input paths
network_paths = []
network_paths.extend(paths.network_output.dropna())
#Specify influence output
influence_paths = []
influence_paths.extend(paths.avg_influence_output.dropna())
#Specify log output
influence_log = []
influence_log.extend(paths.avg_influence_log.dropna())

for network, output_dir, log in zip(network_paths, influence_paths, influence_log):
    sp.check_call(f'nice -15 ananse influence '
                  f'-s /scratch/bacint/jdeleuw/ananse/network_data/avg_network.txt '
                  f'-t {network} '
                  f'-d /scratch/bacint/jdeleuw/ananse/expression_data/DESeq/DEgenes_Day0vsAdultKC.tsv '
                  f'-p '
                  f'-o {output_dir} ' 
                  f'2> {log}',
                  shell = True) 